##### Copyright 2021 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# SessionRunHook を Keras コールバックに移行する

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://www.tensorflow.org/guide/migrate/sessionrunhook_callback"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">TensorFlow.org で表示</a>
</td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/ja/guide/migrate/sessionrunhook_callback.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Google Colab で実行</a>
</td>
  <td><a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/ja/guide/migrate/sessionrunhook_callback.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">GitHub でソースを表示</a></td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/ja/guide/migrate/sessionrunhook_callback.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">ノートブックをダウンロード</a>   </td>
</table>


TensorFlow 1 でトレーニングの動作をカスタマイズするには、`tf.estimator.Estimator` で `tf.estimator.SessionRunHook` を使用します。このガイドでは、`SessionRunHook` から `tf.keras.callbacks.Callback` API を使用して TensorFlow 2 のカスタムコールバックに移行する方法を示します。これは、トレーニングのために Keras `Model.fit`（`Model.evaluate` および `Model.predict` も）と使用できます。この方法を学習するために、トレーニング時に 1 秒あたりのサンプルを測定する `SessionRunHook` と `Callback` タスクを実装します。

コールバックの例は、チェックポイントの保存 （`tf.keras.callbacks.ModelCheckpoint`）と [TensorBoard](%60tf.keras.callbacks.TensorBoard%60) の要約の書き込みです。Keras [コールバック](https://www.tensorflow.org/guide/keras/custom_callback)は、組み込みの Keras `Model.fit`/`Model.evaluate`/`Model.predict` API のトレーニング/評価/予測時にさまざまな時点で呼び出されるオブジェクトです。コールバックの詳細については、`tf.keras.callbacks.Callback` API ドキュメント、および[独自のコールバックの作成](https://www.tensorflow.org/guide/keras/custom_callback.ipynb/)と[組み込みメソッドを使用したトレーニングと評価](https://www.tensorflow.org/guide/keras/train_and_evaluate)（*コールバックの使用*セクション）ガイドを参照してください。

## セットアップ

まず、インポートとデモ用の単純なデータセットから始めます。

In [ ]:
import tensorflow as tf
import tensorflow.compat.v1 as tf1

import time
from datetime import datetime
from absl import flags

In [ ]:
features = [[1., 1.5], [2., 2.5], [3., 3.5]]
labels = [[0.3], [0.5], [0.7]]
eval_features = [[4., 4.5], [5., 5.5], [6., 6.5]]
eval_labels = [[0.8], [0.9], [1.]]

## TensorFlow 1: tf.estimator API を使用してカスタム SessionRunHook を作成する

次の TensorFlow 1 の例は、トレーニング時に 1 秒あたりのサンプルを測定するカスタム `SessionRunHook` を設定する方法を示しています。フック (`LoggerHook`) を作成し、`tf.estimator.Estimator.train` の `hooks` パラメータに渡します。

In [ ]:
def _input_fn():
  return tf1.data.Dataset.from_tensor_slices(
      (features, labels)).batch(1).repeat(100)

def _model_fn(features, labels, mode):
  logits = tf1.layers.Dense(1)(features)
  loss = tf1.losses.mean_squared_error(labels=labels, predictions=logits)
  optimizer = tf1.train.AdagradOptimizer(0.05)
  train_op = optimizer.minimize(loss, global_step=tf1.train.get_global_step())
  return tf1.estimator.EstimatorSpec(mode, loss=loss, train_op=train_op)

In [ ]:
class LoggerHook(tf1.train.SessionRunHook):
  """Logs loss and runtime."""

  def begin(self):
    self._step = -1
    self._start_time = time.time()
    self.log_frequency = 10

  def before_run(self, run_context):
    self._step += 1

  def after_run(self, run_context, run_values):
    if self._step % self.log_frequency == 0:
      current_time = time.time()
      duration = current_time - self._start_time
      self._start_time = current_time
      examples_per_sec = self.log_frequency / duration
      print('Time:', datetime.now(), ', Step #:', self._step,
            ', Examples per second:', examples_per_sec)

estimator = tf1.estimator.Estimator(model_fn=_model_fn)

# Begin training.
estimator.train(_input_fn, hooks=[LoggerHook()])

## TensorFlow 2: Model.fit のカスタム Keras コールバックを作成する

TensorFlow 2 では、組み込みの Keras `Model.fit`（または `Model.evaluate`）をトレーニング/評価に使用する場合、カスタム `tf.keras.callbacks.Callback` を構成し、`Model.fit`（または `Model.evaluate`）の `callbacks` パラメータに渡します。（詳細については、[独自のコールバックの作成](../..guide/keras/custom_callback.ipynb)ガイドを参照してください）。

以下の例では、さまざまな指標をログに記録するカスタム `tf.keras.callbacks.Callback` を記述します。これは 1 秒あたりのサンプルを測定します。これは、前の `SessionRunHook` のサンプルの指標と同様になるはずです。

In [ ]:
class CustomCallback(tf.keras.callbacks.Callback):

    def on_train_begin(self, logs = None):
      self._step = -1
      self._start_time = time.time()
      self.log_frequency = 10

    def on_train_batch_begin(self, batch, logs = None):
      self._step += 1

    def on_train_batch_end(self, batch, logs = None):
      if self._step % self.log_frequency == 0:
        current_time = time.time()
        duration = current_time - self._start_time
        self._start_time = current_time
        examples_per_sec = self.log_frequency / duration
        print('Time:', datetime.now(), ', Step #:', self._step,
              ', Examples per second:', examples_per_sec)

callback = CustomCallback()

dataset = tf.data.Dataset.from_tensor_slices(
    (features, labels)).batch(1).repeat(100)

model = tf.keras.models.Sequential([tf.keras.layers.Dense(1)])
optimizer = tf.keras.optimizers.Adagrad(learning_rate=0.05)

model.compile(optimizer, "mse")

# Begin training.
result = model.fit(dataset, callbacks=[callback], verbose = 0)
# Provide the results of training metrics.
result.history

## 次のステップ

コールバックの詳細については、次を参照してください。

- API ドキュメント: `tf.keras.callbacks.Callback`
- ガイド: [コールバックを記述する](../..guide/keras/custom_callback.ipynb/)
- ガイド: [組み込みメソッドを使用したトレーニングと評価](https://www.tensorflow.org/guide/keras/train_and_evaluate)（*コールバックの使用*セクション）

次の移行関連のリソースも参照してください。

- [早期停止移行ガイド](early_stopping.ipynb): `tf.keras.callbacks.EarlyStopping` は組み込みの早期停止コールバックです
- [TensorBoard 移行ガイド](tensorboard.ipynb): TensorBoard により、指標の追跡と表示が可能になります
- [LoggingTensorHook と StopAtStepHook から Keras コールバックへの移行ガイド](logging_stop_hook.ipynb)